<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="1000" alt="cognitiveclass.ai logo"  />
</center>

# Assignment: Exploratory Data Analysis with SQL

Estimated time needed: 60 minutes

# Introduction

Using this R notebook you will perform exploratory data analysis tasks using SQL queries with the RODBC R package. You will not be graded here, but you will be asked to present some of these results in your final assignment, where you will be graded on the accuracy of your results as well as the content of your SQL queries.


### Establish your Db2 connection

Load the 'RODBC' library, and use the 'odbcConnect( )' function as you did in the previous lab to establish the connection to your Db2 assets.\
Provided you successfully loaded your data into the tables in that lab, you are now ready to start running SQL queries using the RODBC library as you did in Course 3.


In [ ]:
# provide your solution here

In [5]:
library(RODBC)

In [6]:
dsn_driver <- "{IBM DB2 ODBC Driver}"
dsn_database <- "bludb"            # e.g. "bludb"
dsn_hostname <- "19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud" # e.g "54a2f15b-5c0f-46df-8954-.databases.appdomain.cloud"
dsn_port <- "30699"   # e.g. "32733" 
dsn_protocol <- "TCPIP"            # i.e. "TCPIP"
dsn_uid <- "znh14018"        # e.g. "zjh17769"
dsn_pwd <- "zoVWAHZXqFvxz1Se"      # e.g. "zcwd4+8gbq9bm5k4"  
dsn_security <- "ssl"

In [7]:
conn_path <- paste("DRIVER=",dsn_driver,
                  ";DATABASE=",dsn_database,
                  ";HOSTNAME=",dsn_hostname,
                  ";PORT=",dsn_port,
                  ";PROTOCOL=",dsn_protocol,
                  ";UID=",dsn_uid,
                  ";PWD=",dsn_pwd,
                  ";SECURITY=",dsn_security,        
                    sep="")

In [8]:
conn <- odbcDriverConnect(conn_path)

In [9]:
data_type <- sqlTypeInfo(conn)
get.info <- odbcGetInfo(conn)
get.info

DBMS_Name         DBMS_Ver  Driver_ODBC_Ver Data_Source_Name 
"DB2/LINUXX8664"     "11.05.0700"          "03.51"               "" 
     Driver_Name       Driver_Ver         ODBC_Ver      Server_Name 
      "libdb2.a"     "11.05.0600"          "03.52"            "DB2"

## Task 1 - Record Count

#### Determine how many records are in the seoul_bike_sharing dataset.


### Solution 1


In [20]:
# provide your solution here
sqlQuery(conn,"select count(*) from seoul_bike_sharing;")

,1
,<int>
1,8465


## Task 2 - Operational Hours

#### Determine how many hours had non-zero rented bike count.


In [12]:
sqlQuery(conn,"select * from seoul_bike_sharing 
limit 3;")

,DATE,RENTED_BIKE_COUNT,HOUR,TEMPERATURE,HUMIDITY,WIND_SPEED,VISIBILITY,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,SNOWFALL,SEASONS,HOLIDAY,FUNCTIONING_DAY
,<fct>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>
1,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0,0,0,Winter,No Holiday,Yes
2,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0,0,0,Winter,No Holiday,Yes
3,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0,0,0,Winter,No Holiday,Yes


### Solution 2


In [21]:
# provide your solution here
sqlQuery(conn,"select count(HOUR) as count from seoul_bike_sharing
where RENTED_BIKE_COUNT != 0;")

,COUNT
,<int>
1,8465


## Task 3 - Weather Outlook

#### Query the the weather forecast for Seoul over the next 3 hours.

Recall that the records in the CITIES_WEATHER_FORECAST dataset are 3 hours apart, so we just need the first record from the query.


In [24]:
sqlQuery(conn,"select * from CITIES_WEATHER_FORECAST 
limit 3;")

,CITY,WEATHER,VISIBILITY,TEMP,TEMP_MIN,TEMP_MAX,PRESSURE,HUMIDITY,WIND_SPEED,WIND_DEG,SEASON,FORECAST_DATETIME
,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<fct>,<dttm>
1,Seoul,Clear,10000,12.32,10.91,12.32,1015,50,2.18,248,Spring,2021-04-16 12:00:00
2,Seoul,Clear,10000,11.48,9.81,11.48,1016,48,1.25,142,Spring,2021-04-16 15:00:00
3,Seoul,Clouds,10000,9.99,8.82,9.99,1015,46,0.94,130,Spring,2021-04-16 18:00:00


### Solution 3


In [25]:
# provide your solution here
sqlQuery(conn,"select * from CITIES_WEATHER_FORECAST 
limit 1;")

,CITY,WEATHER,VISIBILITY,TEMP,TEMP_MIN,TEMP_MAX,PRESSURE,HUMIDITY,WIND_SPEED,WIND_DEG,SEASON,FORECAST_DATETIME
,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<fct>,<dttm>
1,Seoul,Clear,10000,12.32,10.91,12.32,1015,50,2.18,248,Spring,2021-04-16 12:00:00


## Task 4 - Seasons

#### Find which seasons are included in the seoul bike sharing dataset.


### Solution 4


In [8]:
# provide your solution here
sqlQuery(conn,"select SEASONS from seoul_bike_sharing 
group by SEASONS;")

,SEASONS
,<fct>
1,Autumn
2,Spring
3,Summer
4,Winter


## Task 5 - Date Range

#### Find the first and last dates in the Seoul Bike Sharing dataset.


### Solution 5


In [10]:
# provide your solution here
sqlQuery(conn,"select DATE AS FIRTST_DATE from seoul_bike_sharing 
ORDER BY SEASONS DESC LIMIT 1;")
sqlQuery(conn,"select DATE AS LAST_DATE from seoul_bike_sharing 
ORDER BY SEASONS ASC LIMIT 1;")

,FIRTST_DATE
,<fct>
1,01/12/2017


,LAST_DATE
,<fct>
1,01/09/2018


## Task 6 - Subquery - 'all-time high'

#### determine which date and hour had the most bike rentals.


In [14]:
# provide your solution here
sqlQuery(conn,"select DATE, HOUR,RENTED_BIKE_COUNT from seoul_bike_sharing 
               where RENTED_BIKE_COUNT = (select MAX(RENTED_BIKE_COUNT)
               from seoul_bike_sharing ) ")

,DATE,HOUR,RENTED_BIKE_COUNT
,<fct>,<int>,<int>
1,19/06/2018,18,3556


### Solution 6


## Task 7 - Hourly popularity and temperature by season

#### Determine the average hourly temperature and the average number of bike rentals per hour over each season. List the top ten results by average bike count.


### Solution 7


In [13]:
# provide your solution here
sqlQuery(conn,"select avg(RENTED_BIKE_COUNT),avg(TEMPERATURE)
               from seoul_bike_sharing
               GROUP BY SEASONS
               LIMIT 10;")
               

,1,2
,<int>,<dbl>
1,924,13.821167
2,746,13.021389
3,1034,26.587274
4,225,-2.540463


## Task 8 - Rental Seasonality

#### Find the average hourly bike count during each season.

Also include the minimum, maximum, and standard deviation of the hourly bike count for each season.


### Solution 8


In [16]:
# provide your solution here
sqlQuery(conn,"select avg(RENTED_BIKE_COUNT) AS RENDED_AVG, MIN(RENTED_BIKE_COUNT) AS RENDED_MIN,
               MAX(RENTED_BIKE_COUNT) AS RENDED_MAX, SEASONS from seoul_bike_sharing
               GROUP BY SEASONS;)");

,RENDED_AVG,RENDED_MIN,RENDED_MAX,SEASONS
,<int>,<int>,<int>,<fct>
1,924,2,3298,Autumn
2,746,2,3251,Spring
3,1034,9,3556,Summer
4,225,3,937,Winter


Let's explore a bit and see what might be the most significant contributing factors in terms of the provided data.


CITIES_WEATHER_FORECAST## Task 9 - Weather Seasonality

#### Consider the weather over each season. On average, what were the TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, DEW_POINT_TEMPERATURE, SOLAR_RADIATION, RAINFALL, and SNOWFALL per season?

Include the average bike count as well , and rank the results by average bike count so you can see if it is correlated with the weather at all.


### Solution 9


In [26]:
query <- sqlQuery(conn,"select AVG(TEMPERATURE) AS avg_temp, AVG(HUMIDITY) as avg_humid, AVG(WIND_SPEED) as avg_wind_speed,
    AVG(VISIBILITY) as avg_visibility, AVG(DEW_POINT_TEMPERATURE) as avg_dew_temp, 
    AVG(SOLAR_RADIATION)as avg_solar_rad,AVG(RAINFALL) as avg_rain, AVG(SNOWFALL)as avg_snow,SEASONS 
    from seoul_bike_sharing GROUP BY SEASONS ;")
# provide your solution here

,AVG_TEMP,AVG_HUMID,AVG_WIND_SPEED,AVG_VISIBILITY,AVG_DEW_TEMP,AVG_SOLAR_RAD,AVG_RAIN,AVG_SNOW,SEASONS
,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,13.821167,59,1.492101,1558,5.150594,0.5227827,0.11765617,0.06350026,Autumn
2,13.021389,58,1.857778,1240,4.091389,0.6803009,0.18694444,0.00000000,Spring
3,26.587274,64,1.609420,1501,18.750136,0.7612545,0.25348732,0.00000000,Summer
4,-2.540463,49,1.922685,1445,-12.416667,0.2981806,0.03282407,0.24750000,Winter


In [18]:
query

[1] "42S22 -206 [IBM][CLI Driver][DB2/LINUXX8664] SQL0206N  \"SEASONS\" is not valid in the context where it is used.  SQLSTATE=42703\n"                                                                                                                                                        
[2] "[RODBC] ERROR: Could not SQLExecDirect 'select AVG(TEMPERATURE), AVG(HUMIDITY), AVG(WIND_SPEED),AVG(VISIBILITY),\n                    AVG(DEW_POINT_TEMPERATURE), AVG(SOLAR_RADIATION), AVG(RAINFALL), AVG(SNOWFALL) \n                    from CITIES_WEATHER_FORECAST GROUP BY SEASONS;'"

Seoul## Task 10 - Total Bike Count and City Info for Seoul

#### Use an implicit join across the WORLD_CITIES and the BIKE_SHARING_SYSTEMS tables to determine the total number of bikes avaialble in Seoul, plus the following city information about Seoul: CITY, COUNTRY, LAT, LON, POPULATION, in a single view.

Notice that in this case, the CITY column will work for the WORLD_CITIES table, but in general you would have to use the CITY_ASCII column.


### Solution 10


In [12]:
# provide your solution here                                              
sqlQuery(conn,"select  B.BICYCLES, W.CITY, W.COUNTRY, W.LAT, W.LNG, W.POPULATION 
              from WORLD_CITIES as W ,  BIKE_SHARING_SYSTEMS as B
               where W.CITY_ASCII=B.CITY AND W.CITY_ASCII ='Seoul' ;")

,BICYCLES,CITY,COUNTRY,LAT,LNG,POPULATION
,<int>,<fct>,<fct>,<dbl>,<dbl>,<int>
1,20000,Seoul,"Korea, South",37.58,127,21794000


## Task 11 - Find all city names and coordinates with comparable bike scale to Seoul's bike sharing system

#### Find all cities with total bike counts between 15000 and 20000. Return the city and country names, plus the coordinates (LAT, LNG), population, and number of bicycles for each city.

Later we will ask you to visualize these similar cities on leaflet, with some weather data.


### Solution 11


In [31]:
# provide your solution here
sqlQuery(conn,"select  B.BICYCLES, W.CITY, W.COUNTRY, W.LAT, W.LNG, W.POPULATION 
                from WORLD_CITIES as W ,  BIKE_SHARING_SYSTEMS as B
               where B.BICYCLES  between 15000 and 20000 and  W.CITY_ASCII=B.CITY   ;")

BICYCLES,CITY,COUNTRY,LAT,LNG,POPULATION
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [ ]:
close(conn)

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Yan Luo, Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                  |
| ---------- | ------- | ------------- | ----------------------------------- |
| 2021-05-04 | 0.2     | Jeff Grossman | Finalize                            |
| 2021-04-22 | 0.2     | Jeff Grossman | Update as per peer review           |
| 2021-04-10 | 0.2     | Jeff Grossman | Add data sources and implicit joins |
| 2021-04-05 | 0.1     | Jeff Grossman | Start content creation              |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
